# Reuters Newswire Dataset Trained with Multi-Layer Percentron (Keras)
* Draft: 2020-11-25 (Wed)


## Summary
### Project
* Text classification
* Multiclass classification
  * More specifically, single-label multiclass classification
* Train a 2-layer Multi-Layer Perceptron (MLP) on the Reuters newswire dataset

### Reuters Newswire Dataset
* 11,228 newswires from [Reuters](https://www.reuters.com/)
* labeled over 46 topics; 46 classes
  * The label is an integer ranging from 0 to 45.
* See [7_reuters_newswire_dataset.ipynb](7_reuters_newswire_dataset.ipynb) for details.

## Tutorials
### Official tutorial by Keras
* Not available

### Book chapter
* [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python), François Chollet, Nov. 2017, Manning Publications
  * [3.5. Classifying newswires: a multiclass classification example](https://livebook.manning.com/book/deep-learning-with-python/chapter-3/193)
  
* 케라스 창시자에게 배우는 딥러닝, 프랑소와 숄레 지음, 박해선 옮김, 길벗, 2018-10-22
  * 3.5. 뉴스 기사 분류: 다중 분류 문제, pp.117-126


### Selected Google search results
Keywords: keras reuters example

* [Reuters - Document classification with Keras TF](https://www.kaggle.com/drscarlat/reuters-document-classification-with-keras-tf), Kaggle

## Sample Source Code


### Data collection

In [3]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data( num_words=10000 )

2113536/2110848 [==============================] - 0s 0us/step


#### Double-check

In [4]:
# Double-check the dataset size
print( train_data.shape[0], 'training samples' )
print( test_data.shape[0], 'test samples' )

8982 training samples
2246 test samples


In [5]:
total_sample_size = train_data.shape[0] + test_data.shape[0]
print( total_sample_size, 'total samples' )

11228 total samples


In [6]:
print( train_labels.shape[0], 'training labels,', test_labels.shape[0], 'test labels')

8982 training labels, 2246 test labels


In [7]:
# Double-check the content of the dataset by seeing a sample data
sample_data = train_data[10]

# Notice the sample data is not human-readable.
# The first 10 elemetns
sample_data[:10]

[1, 245, 273, 207, 156, 53, 74, 160, 26, 14]

In [8]:
word_index = reuters.get_word_index()

# Rename the variable name for convenience
word2index_dict = word_index

# Reverse the key-value to value-key for all items of word2index_dict
index2word_dict = dict( [(value, key) for (key, value) in word2index_dict.items()] )

557056/550378 [==============================] - 0s 0us/step


In [9]:
# Let's see what index2word_dict has got.
index2word_dict[1], index2word_dict[245], index2word_dict[273], index2word_dict[207], index2word_dict[156], index2word_dict[53], index2word_dict[74], index2word_dict[160], index2word_dict[26], index2word_dict[14]

('the',
 'federal',
 'gain',
 'only',
 'growth',
 'lt',
 'they',
 'meeting',
 'year',
 'reuter')

0, 1, 2 are padding, start, oov, respectively. So

In [10]:
index2word_dict[245-3], index2word_dict[273-3], index2word_dict[207-3], index2word_dict[156-3], index2word_dict[53-3], index2word_dict[74-3], index2word_dict[160-3], index2word_dict[26-3], index2word_dict[14-3]

('period', 'ended', 'december', '31', 'shr', 'profit', '11', 'cts', 'vs')

In [11]:
# Decode the sample data
# For each word in sample_data, change an index to a word and put a space ' '.
decoded_sample_data = ' '.join( [ index2word_dict.get(word - 3, '?') for word in
sample_data])

print( decoded_sample_data )

? period ended december 31 shr profit 11 cts vs loss 24 cts net profit 224 271 vs loss 511 349 revs 7 258 688 vs 7 200 349 reuter 3


### Data Preparation

In [16]:
# One-hot encoding, https://en.wikipedia.org/wiki/One-hot
# All columns are 0 except 1 for the index.
def vectorize_sequences( sequences, dimension=10000 ):
    # One-hot encoding with 10k columns
    import numpy as np

    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [17]:
# Vectorize the training and test data to tensors with 10k columns
x_train = vectorize_sequences( train_data )
x_test = vectorize_sequences( test_data )

print("x_train", x_train.shape)
print("x_test ", x_test.shape)

x_train (8982, 10000)
x_test  (2246, 10000)


Keras's built-in function `keras.utils.to_categorical` will be used to encode the labels. See below just to deepen the understanding of how one-hot encoding of the labels works.

Example of [One-hot encoding](https://en.wikipedia.org/wiki/One-hot)

| Binary | Gray code | One-hot  |
| :----: | :-------: | :------: |
|  000   |    000    | 00000001 |
|  001   |    001    | 00000010 |
|  010   |    011    | 00000100 |
|  011   |    010    | 00001000 |
|  100   |    110    | 00010000 |
|  101   |    111    | 00100000 |
|  110   |    101    | 01000000 |
|  111   |    100    | 10000000 |

In [25]:
def encode_to_one_hot( labels, dimension=46 ):
    import numpy as np

    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [26]:
y_train_dummy = encode_to_one_hot( train_labels )
y_test_dummy = encode_to_one_hot( test_labels )

print("y_train_dummy", y_train_dummy.shape)
print("y_test_dummy ", y_test_dummy.shape)

y_train_dummy (8982, 46)
y_test_dummy  (2246, 46)


In [29]:
print( train_labels[0], y_train_dummy[0] )
print( train_labels[1], y_train_dummy[1] )

3 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [32]:
# Convert class vectors to binary class matrices
from keras.utils.np_utils import to_categorical

num_classes = 46
y_train = to_categorical( train_labels, num_classes )
y_test = to_categorical( test_labels, num_classes )

In [33]:
print("y_train", y_train.shape)
print("y_test ", y_test.shape)

print( train_labels[0], y_train[0] )
print( train_labels[1], y_train[1] )

y_train (8982, 46)
y_test  (2246, 46)
3 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [38]:
# Setting aside a VALIDATION set
x_val           = x_train[:1000]
partial_x_train = x_train[1000:]
y_val           = y_train[:1000]
partial_y_train = y_train[1000:]

print("x_val ", x_val.shape)
print("y_val ", y_val.shape)

print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

x_val  (1000, 10000)
y_val  (1000, 46)
partial_x_train  (7982, 10000)
partial_y_train  (7982, 46)



## Model Training

We will start off with a basic model for simplicity. Note there are 46 output neurons because 46 classes exist. Given 46 output neurons, 64 input neurons are selected. The size of the input layer is a slightly larger than that of the output layer. The hidden layer size is set to be equal to the input layer.

Say there exists 16 input neurons. The network size is too small.

> In a stack of Dense layers like that you’ve been using, each layer can only access information present in the output of the previous layer. If one layer drops some information relevant to the classification problem, this information can never be recovered by later layers: each layer can potentially become an information bottleneck. In the previous example, you used 16-dimensional intermediate layers, but a 16-dimensional space may be too limited to learn to separate 46 different classes: such small layers may act as information bottlenecks, permanently dropping relevant information. - Source: [3.5.3. Building your network](https://livebook.manning.com/book/deep-learning-with-python/chapter-3/215)


In [39]:
# MODEL
from keras import models
from keras import layers

model = models.Sequential()
model.add( layers.Dense(64, activation='relu', input_shape=(10000,)) )
model.add( layers.Dense(64, activation='relu') )
model.add( layers.Dense(46, activation='softmax') )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                640064    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 46)                2990      
Total params: 647,214
Trainable params: 647,214
Non-trainable params: 0
_________________________________________________________________


The network size used by Kaggle's [Reuters - Document classification with Keras TF](https://www.kaggle.com/drscarlat/reuters-document-classification-with-keras-tf):

```python
model = models.Sequential()
model.add( layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(10000,)) )
model.add( layers.Dropout(0.5) )
model.add( layers.Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu') )
model.add( layers.Dropout(0.5) )
model.add( layers.Dense(46, activation='softmax') )
model.summary()
```

```python
# REGULARIZERS L1 L2
#regularizers.l1(0.001)
#regularizers.l2(0.001)
#regularizers.l1_l2(l1=0.001, l2=0.001)

# Best results I got with HU=128/128/128 or 256/256 and L1=0.001 and Dropout=0.5 = 77.02%
# Without Regularizer 72.92%
# Reg L1 = 76.04, L2 = 76.2, L1_L2 = 76.0
# Only DropOut (0.5) = 76.85%
```


In [40]:
NumEpochs = 10
BatchSize = 512

model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(partial_x_train, partial_y_train, epochs=NumEpochs, batch_size=BatchSize, validation_data=(x_val, y_val))

Epoch 1/10
16/16 [==============================] - 0s 23ms/step - loss: 2.5841 - accuracy: 0.5182 - val_loss: 1.7273 - val_accuracy: 0.6310
Epoch 2/10
16/16 [==============================] - 0s 13ms/step - loss: 1.4152 - accuracy: 0.7060 - val_loss: 1.2869 - val_accuracy: 0.7140
Epoch 3/10
16/16 [==============================] - 0s 14ms/step - loss: 1.0544 - accuracy: 0.7734 - val_loss: 1.1214 - val_accuracy: 0.7480
Epoch 4/10
16/16 [==============================] - 0s 13ms/step - loss: 0.8293 - accuracy: 0.8232 - val_loss: 1.0712 - val_accuracy: 0.7510
Epoch 5/10
16/16 [==============================] - 0s 14ms/step - loss: 0.6594 - accuracy: 0.8611 - val_loss: 0.9689 - val_accuracy: 0.7950
Epoch 6/10
16/16 [==============================] - 0s 14ms/step - loss: 0.5264 - accuracy: 0.8898 - val_loss: 0.9044 - val_accuracy: 0.8090
Epoch 7/10
16/16 [==============================] - 0s 14ms/step - loss: 0.4225 - accuracy: 0.9105 - val_loss: 0.9144 - val_accuracy: 0.8090
Epoch 8/10
16

TODO: https://www.kaggle.com/drscarlat/reuters-document-classification-with-keras-tf

Start from 

decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])